In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('charity_navigator_clean.csv', index_col = 0)

In [3]:
n = 0
for excess_or_deficit in df['excess_or_deficit_for_year']:
    if df.loc[n, 'excess_or_deficit_for_year'] > 0:
        df.loc[n, 'excess_or_deficit'] = 1
        n += 1
    elif df.loc[n, 'excess_or_deficit_for_year'] <= 0:
        df.loc[n, 'excess_or_deficit'] = 0
        n += 1
    else:
        n += 1

In [4]:
comp_leader_df = df[df['compensation_leader_compensation'].isnull() == False]
org_type_dummies = pd.get_dummies(comp_leader_df.org_type)
comp_leader_df = pd.concat([comp_leader_df, org_type_dummies], axis=1, join = 'outer')
comp_leader_df.drop(['charity_name', 'charity_url', 
                     'city', 'cn_advisory', 
                     'state', 'org_type',
                     'org_category', 'compensation_leader_title'], 
                    axis = 1, inplace = True)
comp_leader_df.head()

,accountability_score,administrative_expenses,compensation_leader_compensation,compensation_leader_expense_percent,excess_or_deficit_for_year,financial_score,fundraising_expenses,net_assets,other_revenue,overall_score,...,"Arts, Culture, Humanities",Community Development,Education,Environment,Health,Human Services,Human and Civil Rights,International,Religion,Research and Public Policy
0,89.0,164590.0,53463.0,3.11,349718.0,90.56,111522,1350382,49634.0,89.75,...,0,0,0,0,0,0,1,0,0,0
1,86.0,1001560.0,73500.0,1.47,1175965.0,85.92,54613,14773920,382540.0,85.95,...,0,0,1,0,0,0,0,0,0,0
2,85.0,93957.0,85000.0,0.99,-461502.0,77.65,248833,-770370,0.0,80.96,...,0,0,0,0,0,0,0,1,0,0
3,86.0,346867.0,61220.0,0.78,1872733.0,97.50,384550,11460087,-81726.0,89.94,...,0,0,1,0,0,0,0,0,0,0
4,97.0,135195.0,74244.0,5.41,-103940.0,87.08,87436,723772,32436.0,90.62,...,0,0,0,0,0,0,0,0,1,0


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [6]:
scaler = StandardScaler()
scaler.fit(comp_leader_df.drop('excess_or_deficit', axis = 1))
scaled_features = scaler.transform(comp_leader_df.drop('excess_or_deficit', axis = 1))
df_scaled_features = pd.DataFrame(scaled_features, columns = comp_leader_df.columns[:-1])
df_scaled_features.head()

,accountability_score,administrative_expenses,compensation_leader_compensation,compensation_leader_expense_percent,excess_or_deficit_for_year,financial_score,fundraising_expenses,net_assets,other_revenue,overall_score,...,Animals,"Arts, Culture, Humanities",Community Development,Education,Environment,Health,Human Services,Human and Civil Rights,International,Religion
0,-0.263579,-0.186773,-0.866057,-0.181814,-0.047114,0.462148,-0.156168,-0.176026,-0.146236,0.330661,...,-0.417255,-0.327716,-0.285942,-0.230243,-0.334349,-0.635888,4.886773,-0.270780,-0.229912,-0.157289
1,-0.601692,-0.048412,-0.711788,-0.869707,0.022858,-0.004671,-0.165434,-0.099415,-0.103574,-0.157966,...,-0.417255,-0.327716,3.497212,-0.230243,-0.334349,-0.635888,-0.204634,-0.270780,-0.229912,-0.157289
2,-0.714396,-0.198450,-0.623248,-1.071041,-0.115813,-0.836696,-0.133811,-0.188129,-0.152597,-0.799611,...,-0.417255,-0.327716,-0.285942,-0.230243,-0.334349,-0.635888,-0.204634,3.693041,-0.229912,-0.157289
3,-0.601692,-0.156641,-0.806334,-1.159125,0.081865,1.160365,-0.111714,-0.118328,-0.163070,0.355093,...,-0.417255,-0.327716,3.497212,-0.230243,-0.334349,-0.635888,-0.204634,-0.270780,-0.229912,-0.157289
4,0.638054,-0.191633,-0.706060,0.782913,-0.085533,0.112034,-0.160089,-0.179602,-0.148440,0.442531,...,-0.417255,-0.327716,-0.285942,-0.230243,-0.334349,-0.635888,-0.204634,-0.270780,4.349486,-0.157289


In [7]:
X = df_scaled_features[['accountability_score', 'financial_score',
                        'overall_score', 'administrative_expenses',
                        'fundraising_expenses', 'net_assets',
                        'other_revenue', 'payments_to_affiliates',
                        'compensation_leader_compensation', 
                        'compensation_leader_expense_percent',
                        'Animals', 'Arts, Culture, Humanities',
                        'Community Development', 'Education',
                        'Environment', 'Health', 'Human Services',
                        'Human and Civil Rights', 'International',
                        'Religion']]
y = comp_leader_df['excess_or_deficit']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 101)
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

In [9]:
print('R^2: ', dtree.score(X_train, y_train).round(4))
print('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(4))
print('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(4))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(4))

R^2:  1.0
MAE:  0.4698
MSE:  0.4698
RMSE: 0.6854


In [10]:
print(metrics.classification_report(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.45      0.46      0.46      1245
        1.0       0.59      0.58      0.59      1671

avg / total       0.53      0.53      0.53      2916

[[576 669]
 [701 970]]


In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
rfc = RandomForestClassifier(n_estimators = 600)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [13]:
print(metrics.classification_report(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.51      0.33      0.40      1245
        1.0       0.60      0.76      0.67      1671

avg / total       0.56      0.58      0.56      2916

[[ 414  831]
 [ 402 1269]]
